# Case Study 1 : Collecting Data from Twitter

Due Date: 2/8/2021, **BEFORE the beginning of class at 11:00am**

## **NOTE: There are *always* last minute issues submitting the case studies.  DO NOT WAIT UNTIL THE LAST MINUTE!**

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    member 1
    
    member 2
    
    ...

**Suggested Readings:** 
* Chapter 1 and Chapter 9 of the book "Mining the Social Web" can help a lot if you get stuck. 
* In fact, it is intentional that many of these questions can be answered directly from there (except for question 4)!
* The idea is to ease you into the case studies :-)

**Don't forget!**
* You will need to install the python-twitter library to access the Twitter API
 * pip install twitter


** NOTE **
* **Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost**.

# Problem 1 (20 points): Sampling Twitter Data with either Search or Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter API to sample a collection of tweets about this topic. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [2]:
import twitter
import json
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Prof. Paffenroth has a developer account for the class.  He will provide the Twitter access tokens for
    # each team
    # See https://developer.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'oVek8jmhI4sz8SbKnT1e6UsIh'
    CONSUMER_SECRET ='3eGB6ip82K3XowIDEmz9JXIbRc2uFSlHvjPJ8Lmkf3HVIdlmhM'
    OAUTH_TOKEN = '571213367-H0r2WfySE7L4vgUJHv4YLW2vZGLa7TEPP7iWsTNu'
    OAUTH_TOKEN_SECRET = 'uKcYixkDRaCUvaP2oqH9a9fIaIB9nf4NKz32iqo2t7B7K'

    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

twitter_api = oauth_login()

#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

q = 'outage' #key word

count = 100 # change to change how many tweets you get

# Import unquote to prevent URL encoding errors in next_results
from urllib.parse import unquote

search_results = twitter_api.search.tweets(q=q, count=count)

statuses = search_results['statuses']

for _ in range(5): #takes count 5 times, 100*5 = 500
    print('Length of statuses', len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError as e: # No more results when next_results doesn't exist
        break

    # Create a dictionary from next_results, which has the following form:
    kwargs = dict([ kv.split('=') for kv in unquote(next_results[1:]).split("&") ])

    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

# Show one sample search result by slicing the list...
print(json.dumps(statuses[0], indent=1)) 



Length of statuses 100
Length of statuses 200
Length of statuses 300
Length of statuses 400
Length of statuses 500
{
 "created_at": "Fri Feb 05 16:58:59 +0000 2021",
 "id": 1357735483313430528,
 "id_str": "1357735483313430528",
 "text": "RT @akskhshbu: UN observers are not allowed in the valley. Problems in the valley due to internet outage cannot be highlighted, only files\u2026",
 "truncated": false,
 "entities": {
  "hashtags": [],
  "symbols": [],
  "user_mentions": [
   {
    "screen_name": "akskhshbu",
    "name": "S H A H B A Z",
    "id": 1165355895179624449,
    "id_str": "1165355895179624449",
    "indices": [
     3,
     13
    ]
   }
  ],
  "urls": []
 },
 "metadata": {
  "iso_language_code": "en",
  "result_type": "recent"
 },
 "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
 "in_reply_to_status_id": null,
 "in_reply_to_status_id_str": null,
 "in_reply_to_user_id": null,
 "in_reply_to_user_id_str": null,
 "in_reply_to_

### Report some statistics about the tweets you collected 

* The topic of interest: < INSERT YOUR TOPIC HERE>


* The total number of tweets collected:  < INSERT THE NUMBER HERE>

*-----------------------

# Problem 2 (20 points): Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [116]:
import cloudpickle
import nltk
# import matplotlib.pyplot as plt
from prettytable import PrettyTable

# pickling/saving data
f = open('outages.pickle', 'wb')
cloudpickle.dump(statuses, f)
f.close()

# getting only the words in the tweets, clean them
words = []
result_set = set()

for tweet in statuses:
    text = tweet['text']
    text = text.lower()
    dirty_words = [ w for w in text.split() ]
    dirty_words = [word.strip('.,!;()[]:') for word in dirty_words]
    dirty_words = [word.replace("'s", '') for word in dirty_words]
    words += dirty_words

# case insensitive
def no_dupes(data):
    seen, result = set(), []

    for item in data:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result

# frequency distribution
freq_dist = nltk.FreqDist(words)

# top 30 words
top30 = freq_dist.most_common(30)

# table of top 30 words
x = PrettyTable()
x.title = 'Top 30 Words'
x.field_names = ['Word', 'Count']
x.add_rows(top30)
print(x)

+-----------------------+
|      Top 30 Words     |
+---------------+-------+
|      Word     | Count |
+---------------+-------+
|     outage    |  516  |
| @citypowerjhb |  301  |
|       in      |  249  |
|      the      |  245  |
|     power     |  217  |
|       is      |  187  |
|       a       |  153  |
|       to      |  149  |
|       we      |  136  |
|       rt      |  132  |
|       an      |  109  |
|      are      |  105  |
|      you      |  104  |
|      and      |  104  |
|      for      |  103  |
|     please    |   91  |
|       of      |   88  |
|       at      |   86  |
|       on      |   83  |
|      area     |   83  |
|      not      |   82  |
|      this     |   75  |
|  loadshedding |   66  |
|       it      |   54  |
|      load     |   52  |
|      our      |   51  |
|   affecting   |   49  |
|    shedding   |   47  |
|      have     |   42  |
|      that     |   41  |
+---------------+-------+


**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [117]:
def get_tweets_rt_count(data):
    tweets = []
    rt_counts = []

    for tweet in data:
        tweets.append(tweet['text'])
        rt_counts.append(tweet['retweet_count'])

    return list(set(zip(tweets, rt_counts)))

In [118]:
def rt_count(tup):
    return tup[1]

In [119]:
from tables import *
import numpy as np

# retrieve data/unpickle
pickle_off = open("outages.pickle","rb")
outages = cloudpickle.load(pickle_off)

# get only the tweets and retweet count
tweets_rts = get_tweets_rt_count(outages)

# sort by retweet count
tweets_rts.sort(reverse=True,key=rt_count)

# top 10 table
top10 = tweets_rts[:10]

x = PrettyTable()
x.title = 'Top 10 Most Popular Tweets'
x.field_names = ['Tweet', 'Retweet Count']
x.add_rows(top10)
x._max_width = {"Tweet" : 50, "Retweet Count" : 25}
x.hrules = 1
print(x)

+--------------------------------------------------------------------+
|                     Top 10 Most Popular Tweets                     |
+----------------------------------------------------+---------------+
|                       Tweet                        | Retweet Count |
+----------------------------------------------------+---------------+
|  RT @DaKidGowie: To this day, no one can convince  |      477      |
| me that the power outage wasn't the NFL's attempt  |               |
|          to kill the Ravens' momentum. 😒          |               |
+----------------------------------------------------+---------------+
|     RT @BvueFD: CenturyLink is reporting a 911     |       37      |
| landline outage affecting as many as 32,000 homes  |               |
|  in Bellevue. If you need to report an emergenc…   |               |
+----------------------------------------------------+---------------+
|           RT @coinbase: On January 29th,           |       35      |
|   htt

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [107]:
help(PrettyTable)

Help on class PrettyTable in module prettytable.prettytable:

class PrettyTable(builtins.object)
 |  PrettyTable(field_names=None, **kwargs)
 |  
 |  Methods defined here:
 |  
 |  __getattr__(self, name)
 |  
 |  __getitem__(self, index)
 |  
 |  __init__(self, field_names=None, **kwargs)
 |      Return a new PrettyTable instance
 |      
 |      Arguments:
 |      
 |      encoding - Unicode encoding scheme used to decode any encoded input
 |      title - optional table title
 |      field_names - list or tuple of field names
 |      fields - list or tuple of field names to include in displays
 |      start - index of first data row to include in output
 |      end - index of last data row to include in output PLUS ONE (list slice style)
 |      header - print a header showing field names (True or False)
 |      header_style - stylisation to apply to field names in header
 |          ("cap", "title", "upper", "lower" or None)
 |      border - print a border around the table (True or 

In [4]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
#Katie/Bryson

# Problem 3 (20 points): Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [5]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

#Bryson/Katie

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [6]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

#Bryson/Katie

*------------------------

# Problem 4 (20 points): Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [9]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary





*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 15 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu).

#### We auto-process the submissions so make sure your subject line is *exactly*:

### DS3010 Case Study 1 Team ??

#### where ?? is your team number.
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

**Totoal Points: 100**


---------------------------------------------------------------------------
**Notebook results:**
    Points: 80


    -----------------------------------
    Question 1:
    Points: 20
    -----------------------------------
    
    -----------------------------------
    Question 2:
    Points: 20
    -----------------------------------
        
    -----------------------------------
    Question 3:
    Points: 20
    -----------------------------------
  
    -----------------------------------
    Question 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Come up with a business question and describe how Twitter data and a NoSQL database such as MongoDB call help you anser it.


---------------------------------------------------------------------------
**Slides (for 10 minutes of presentation): Story-telling**
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 


